<a href="https://colab.research.google.com/github/thuh66271-arch/TRI-TUE-NHA-TAO1/blob/main/Bai_bao_cao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# BÀI SỐ 1
import heapq
import math
from typing import List, Tuple, Dict, Set, Optional

# --- 1. Định nghĩa Lớp Trạng thái (TSPNode) ---

class TSPNode:
    """
    Đại diện cho một trạng thái (Node) trong quá trình tìm kiếm A* cho TSP.
    """
    def __init__(self, path: List[int], cost: float, current_city: int, unvisited_cities: Set[int]):
        self.path = path
        self.cost = cost
        self.current_city = current_city
        self.unvisited_cities = unvisited_cities

    # Cần định nghĩa các hàm so sánh, băm và bằng để sử dụng trong heapq và dict
    def __lt__(self, other: 'TSPNode') -> bool:
        return self.cost < other.cost

    def __hash__(self) -> int:
        unvisited_tuple = tuple(sorted(list(self.unvisited_cities)))
        return hash((self.current_city, unvisited_tuple))

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, TSPNode):
            return NotImplemented
        return (self.current_city == other.current_city and
                self.unvisited_cities == other.unvisited_cities)


# --- 2. Định nghĩa Lớp Thuật Toán A* (TSP_AStar) ---

class TSP_AStar:
    """
    Triển khai thuật toán A* để tìm chu trình TSP ngắn nhất.
    """
    def __init__(self, distance_matrix: Dict[int, Dict[int, float]]):
        self.dist_matrix = distance_matrix
        self.num_cities = len(distance_matrix)
        self.cities = list(range(self.num_cities))
        self.min_edges = self._calculate_min_edges()

    def _calculate_min_edges(self) -> Dict[int, float]:
        """Tính cạnh ngắn nhất rời khỏi mỗi thành phố cho Heuristic."""
        min_edges = {}
        for i in self.cities:
            min_dist = float('inf')
            for j in self.cities:
                if i != j and self.dist_matrix[i][j] < min_dist:
                    min_dist = self.dist_matrix[i][j]
            min_edges[i] = min_dist
        return min_edges

    def _heuristic(self, node: TSPNode, start_city: int) -> float:
        """
        Hàm Heuristic h(n) (ước lượng chi phí còn lại - Cận Dưới).
        """
        if not node.unvisited_cities:
            # Nếu đã đi hết, chi phí còn lại là quay về điểm xuất phát
            return self.dist_matrix[node.current_city][start_city]

        h_val = 0.0

        # Tổng các cạnh ngắn nhất của các thành phố chưa ghé thăm
        for city in node.unvisited_cities:
            h_val += self.min_edges[city]

        # Cạnh ngắn nhất để quay về thành phố xuất phát (start_city)
        # Ta lấy cạnh từ current_city đến start_city
        h_val += self.dist_matrix[node.current_city][start_city]

        return h_val

    def solve(self, start_city: int = 0) -> Tuple[Optional[float], Optional[List[int]]]:
        """
        Thực thi thuật toán A*. Trả về (tổng chi phí, đường đi).
        """
        # pq: (f_cost, g_cost, node)
        pq: List[Tuple[float, float, TSPNode]] = []
        visited: Dict[TSPNode, float] = {}

        # Khởi tạo
        unvisited_start = set(self.cities)
        unvisited_start.remove(start_city)
        start_node = TSPNode(path=[start_city], cost=0.0, current_city=start_city, unvisited_cities=unvisited_start)

        f_start = start_node.cost + self._heuristic(start_node, start_city)
        heapq.heappush(pq, (f_start, start_node.cost, start_node))
        visited[start_node] = start_node.cost

        best_path: Optional[List[int]] = None
        min_cost: float = float('inf')

        while pq:
            f_cost, g_cost, current_node = heapq.heappop(pq)

            if f_cost >= min_cost:
                continue

            # Kiểm tra mục tiêu: Đã ghé thăm tất cả?
            if not current_node.unvisited_cities:
                return_cost = self.dist_matrix[current_node.current_city][start_city]
                total_cost = current_node.cost + return_cost

                if total_cost < min_cost:
                    min_cost = total_cost
                    best_path = current_node.path + [start_city]

                continue

            # Mở rộng Node
            for next_city in current_node.unvisited_cities:
                transition_cost = self.dist_matrix[current_node.current_city][next_city]
                new_cost = current_node.cost + transition_cost

                new_path = current_node.path + [next_city]
                new_unvisited = current_node.unvisited_cities - {next_city}

                new_node = TSPNode(path=new_path, cost=new_cost, current_city=next_city, unvisited_cities=new_unvisited)

                # Kiểm tra trạng thái đã ghé thăm
                if new_node in visited and visited[new_node] <= new_cost:
                    continue

                visited[new_node] = new_cost
                h_cost = self._heuristic(new_node, start_city)
                f_new = new_cost + h_cost

                if f_new < min_cost:
                    heapq.heappush(pq, (f_new, new_cost, new_node))

        return min_cost, best_path

# --- 3. Hàm Xử lý Nhập Liệu ---

def get_user_input_distance_matrix() -> Optional[Dict[int, Dict[int, float]]]:
    """
    Xử lý nhập liệu từ người dùng để tạo ma trận khoảng cách.
    """
    while True:
        try:
            n_input = input("Nhập số lượng thành phố (N): ")
            num_cities = int(n_input)
            if num_cities < 2:
                print("Lỗi: Số lượng thành phố phải lớn hơn hoặc bằng 2.")
                continue
            break
        except ValueError:
            print("Lỗi: Vui lòng nhập một số nguyên hợp lệ.")

    distance_matrix: Dict[int, Dict[int, float]] = {i: {} for i in range(num_cities)}

    print("\n--- Nhập khoảng cách giữa các thành phố ---")
    print("Khoảng cách từ A -> B có thể khác B -> A (Đồ thị có hướng)")
    print("Thành phố được đánh số từ 0 đến N-1.")

    for i in range(num_cities):
        distance_matrix[i][i] = 0.0 # Khoảng cách từ i đến i là 0

        for j in range(num_cities):
            if i == j:
                continue

            while True:
                try:
                    prompt = f"Khoảng cách từ thành phố {i} đến thành phố {j}: "
                    dist_input = input(prompt)
                    distance = float(dist_input)
                    if distance < 0:
                        print("Lỗi: Khoảng cách không được là số âm. Vui lòng nhập lại.")
                        continue
                    distance_matrix[i][j] = distance
                    break
                except ValueError:
                    print("Lỗi: Vui lòng nhập một giá trị số hợp lệ.")

    return distance_matrix

def run_tsp_astar_example():
    """Chạy chương trình TSP A* với dữ liệu nhập từ người dùng."""
    print("=========================================================")
    print("  Bài Toán Người Giao Hàng (TSP) bằng Thuật Toán A* ")
    print("=========================================================")

    distance_matrix = get_user_input_distance_matrix()

    if not distance_matrix:
        print("Chương trình kết thúc do không có dữ liệu.")
        return

    num_cities = len(distance_matrix)
    print("\n--- Ma trận khoảng cách đã nhập ---")
    print("  " + " | ".join(f"{i:2}" for i in range(num_cities)))
    print("-" * (num_cities * 5))
    for i in range(num_cities):
        row_str = f"{i} | " + " | ".join(f"{distance_matrix[i][j]:.1f}" for j in range(num_cities))
        print(row_str)

    tsp_solver = TSP_AStar(distance_matrix)

    # Mặc định bắt đầu từ thành phố 0
    start_city = 0
    print(f"\n--- Bắt đầu Giải Thuật Toán A* (từ thành phố {start_city}) ---")

    min_cost, best_path = tsp_solver.solve(start_city=start_city)

    print("\n--- KẾT QUẢ ---")
    if best_path:
        path_str = " -> ".join(map(str, best_path))
        print(f"✅ Chu trình ngắn nhất tìm được:")
        print(f"   Đường đi: {path_str}")
        print(f"   Tổng chi phí (độ dài): {min_cost:.2f}")
    else:
        print("❌ Không tìm thấy chu trình hợp lệ.")

    print("=========================================================")


# --- Chạy Chương trình ---
if __name__ == "__main__":
    # Lưu ý: A* giải TSP là một bài toán NP-hard.
    # Với số lượng thành phố lớn (N > 10-12), chương trình có thể chạy RẤT chậm hoặc hết bộ nhớ.
    run_tsp_astar_example()

  Bài Toán Người Giao Hàng (TSP) bằng Thuật Toán A* 
Nhập số lượng thành phố (N): 3

--- Nhập khoảng cách giữa các thành phố ---
Khoảng cách từ A -> B có thể khác B -> A (Đồ thị có hướng)
Thành phố được đánh số từ 0 đến N-1.
Khoảng cách từ thành phố 0 đến thành phố 1: 5
Khoảng cách từ thành phố 0 đến thành phố 2: 8
Khoảng cách từ thành phố 1 đến thành phố 0: 3
Khoảng cách từ thành phố 1 đến thành phố 2: 1
Khoảng cách từ thành phố 2 đến thành phố 0: 2
Khoảng cách từ thành phố 2 đến thành phố 1: 3

--- Ma trận khoảng cách đã nhập ---
   0 |  1 |  2
---------------
0 | 0.0 | 5.0 | 8.0
1 | 3.0 | 0.0 | 1.0
2 | 2.0 | 3.0 | 0.0

--- Bắt đầu Giải Thuật Toán A* (từ thành phố 0) ---

--- KẾT QUẢ ---
✅ Chu trình ngắn nhất tìm được:
   Đường đi: 0 -> 1 -> 2 -> 0
   Tổng chi phí (độ dài): 8.00


In [ ]:
# BÀI SỐ 2
import random
from collections import defaultdict

# --- 1. Cấu hình ban đầu ---
# Giả lập dữ liệu cho bài toán
DAYS = ["Thứ 2", "Thứ 3", "Thứ 4", "Thứ 5", "Thứ 6"]
HOURS = list(range(1, 6))  # Tiết 1 đến Tiết 5
SLOTS_PER_DAY = len(HOURS)
TOTAL_SLOTS = len(DAYS) * SLOTS_PER_DAY # 25 slots

# Yêu cầu số tiết mỗi môn cho MỘT LỚP
COURSE_REQUIREMENTS = {
    "Toán": 4,
    "Văn": 4,
    "Anh": 3,
}
# Giả lập danh sách giáo viên
TEACHERS = {
    "Toán": ["GV_Toán_A", "GV_Toán_B"],
    "Văn": ["GV_Văn_C", "GV_Văn_D"],
    "Anh": ["GV_Anh_E"],
}
# Danh sách lớp học
CLASSES = ["10A", "10B"]
ALL_COURSES = list(COURSE_REQUIREMENTS.keys())

# --- 2. Định nghĩa Chromosome (Cá thể) ---
# Một cá thể là một lịch trình hoàn chỉnh cho TẤT CẢ các lớp.

class Schedule:
    """
    Đại diện cho một Cá thể (Chromosome) trong Giải thuật Di truyền, tức là một Lịch trình hoàn chỉnh.
    """
    def __init__(self):
        """
        Khởi tạo một lịch trình mới.

        self.schedule_data: Dictionary ánh xạ slot_id (0-24) tới danh sách các bài giảng
                           dưới dạng tuple (Lớp, Môn, Giáo viên).
        self.fitness: Độ thích nghi của lịch trình (càng cao càng tốt).
        self.hard_constraint_penalty: Số lần vi phạm ràng buộc cứng (mục tiêu là 0).
        """
        self.schedule_data = defaultdict(list)
        self.fitness = 0.0
        self.hard_constraint_penalty = 0
        self.generate_initial_schedule()

    def generate_initial_schedule(self):
        """
        Khởi tạo một lịch trình ngẫu nhiên hợp lệ sơ bộ bằng cách tạo ra
        tất cả các gen (tiết học) cần thiết và phân phối ngẫu nhiên chúng vào các slot thời gian.
        """

        # 1. Tạo danh sách tất cả các gen (Lớp, Môn, Giáo viên)
        all_genes = []
        for class_name in CLASSES:
            for course, required_slots in COURSE_REQUIREMENTS.items():
                teachers_list = TEACHERS[course]
                for _ in range(required_slots):
                    teacher = random.choice(teachers_list)
                    all_genes.append((class_name, course, teacher))

        # Phân loại gen theo lớp
        class_genes = defaultdict(list)
        for g in all_genes:
             class_genes[g[0]].append(g)

        # 2. Gán Gen vào Slot cho từng lớp
        for class_name in CLASSES:
            # Chọn ngẫu nhiên các slot trong tuần để xếp tiết
            num_lessons = len(class_genes[class_name])
            slots_for_class = random.sample(range(TOTAL_SLOTS), num_lessons)

            for i, slot_id in enumerate(slots_for_class):
                 # Thêm gen (Lớp, Môn, GV) vào slot tương ứng
                 self.schedule_data[slot_id].append(class_genes[class_name][i])


    def calculate_fitness(self):
        """
        Tính toán độ thích nghi (Fitness) của lịch trình dựa trên:
        1. Ràng buộc Cứng (Hard Constraints - Phạt nặng): Trùng lịch GV, trùng lịch Lớp.
        2. Ràng buộc Mềm (Soft Constraints - Phạt nhẹ): Phân bố tiết học, giờ trống GV.

        Nếu vi phạm cứng, fitness giảm mạnh. Nếu không vi phạm cứng, fitness được tính
        dựa trên mức độ tối ưu các ràng buộc mềm.
        """

        # Phân tích lịch trình thành cấu trúc dễ tính toán (Lịch GV, Lịch Lớp)
        teacher_schedule = defaultdict(lambda: [[] for _ in range(TOTAL_SLOTS)])
        class_schedule = defaultdict(lambda: [[] for _ in range(TOTAL_SLOTS)])

        for slot_id, genes in self.schedule_data.items():
            for class_name, course, teacher in genes:
                teacher_schedule[teacher][slot_id].append((class_name, course))
                class_schedule[class_name][slot_id].append((course, teacher))

        # --- RÀNG BUỘC CỨNG (Hard Constraints) ---
        penalty = 0

        # 1. Trùng lịch Giáo viên
        for teacher, slots in teacher_schedule.items():
            for lessons in slots:
                if len(lessons) > 1:
                    penalty += len(lessons) - 1

        # 2. Trùng lịch Lớp học
        for class_name, slots in class_schedule.items():
            for lessons in slots:
                if len(lessons) > 1:
                    penalty += len(lessons) - 1

        self.hard_constraint_penalty = penalty

        # Nếu vi phạm ràng buộc cứng, fitness rất thấp
        if penalty > 0:
            self.fitness = 1.0 / (1.0 + penalty * 1000)
            return

        # --- RÀNG BUỘC MỀM (Soft Constraints) ---
        soft_score = 0

        # 3. Phân bố tiết học môn (Không quá 2 tiết cùng một môn trong 1 ngày)
        for class_name, slots in class_schedule.items():
            for day_index in range(len(DAYS)):
                day_lessons = slots[day_index * SLOTS_PER_DAY : (day_index + 1) * SLOTS_PER_DAY]

                daily_course_count = defaultdict(int)
                for lessons in day_lessons:
                    if lessons:
                         daily_course_count[lessons[0][0]] += 1

                for count in daily_course_count.values():
                    if count > 2:
                        soft_score += (count - 2) * 2

        # 4. Tiết trống của GV (Giảm thiểu giờ rảnh rỗi giữa các tiết dạy)
        teacher_idle_slots = 0
        for teacher, slots in teacher_schedule.items():
            for day_index in range(len(DAYS)):
                day_slots = slots[day_index * SLOTS_PER_DAY : (day_index + 1) * SLOTS_PER_DAY]

                teaching_hours = [i for i, lessons in enumerate(day_slots) if lessons]

                if len(teaching_hours) > 1:
                    start = teaching_hours[0]
                    end = teaching_hours[-1]

                    idle_slots = (end - start + 1) - len(teaching_hours)
                    teacher_idle_slots += idle_slots

        soft_score += teacher_idle_slots * 0.5

        # Tính toán Fitness cuối cùng
        self.fitness = 1.0 / (1.0 + soft_score)

    def display(self):
        """
        In lịch trình tối ưu ra màn hình dưới dạng bảng dễ đọc cho từng lớp học.
        """
        print("--- Lịch trình Tối ưu ---")

        for class_name in CLASSES:
            print(f"\n### Lớp: {class_name}")

            # Tạo bảng lịch cho lớp
            timetable = {day: ["(Trống)"] * SLOTS_PER_DAY for day in DAYS}

            for slot_id in range(TOTAL_SLOTS):
                day_index = slot_id // SLOTS_PER_DAY
                hour_index = slot_id % SLOTS_PER_DAY

                for class_in_slot, course, teacher in self.schedule_data[slot_id]:
                    if class_in_slot == class_name:
                         timetable[DAYS[day_index]][hour_index] = f"{course} ({teacher})"

            # In ra bảng (dùng markdown table)
            header = "| Tiết | " + " | ".join([f"**{d}**" for d in DAYS]) + " |"
            separator = "| :--- |" + " :---: |" * len(DAYS)
            print(header)
            print(separator)

            for hour in HOURS:
                row = f"| **{hour}** | "
                row += " | ".join([timetable[d][hour-1] for d in DAYS])
                print(row + " |")

# --- 3. Các Hàm của Giải thuật Di truyền ---
class GeneticAlgorithm:
    """
    Class quản lý quá trình tiến hóa của Giải thuật Di truyền.
    """
    def __init__(self, population_size: int, generations: int):
        """
        Khởi tạo Bộ giải GA.
        :param population_size: Số lượng cá thể trong quần thể.
        :param generations: Số thế hệ tiến hóa tối đa.
        """
        self.population_size = population_size
        self.generations = generations
        self.population = []
        self.best_schedule = None

    def initialize_population(self):
        """
        Khởi tạo quần thể ban đầu với số lượng cá thể đã định.
        """
        for _ in range(self.population_size):
            self.population.append(Schedule())

    def select_parents(self):
        """
        Chọn lọc cha mẹ bằng phương pháp Tournament Selection.
        Chọn ngẫu nhiên một nhóm, cá thể có fitness cao nhất trong nhóm sẽ được chọn.
        :return: Tuple (parent1, parent2)
        """
        tournament_size = 5

        def tournament():
            candidates = random.sample(self.population, tournament_size)
            return max(candidates, key=lambda s: s.fitness)

        parent1 = tournament()
        parent2 = tournament()
        return parent1, parent2

    def crossover(self, parent1: Schedule, parent2: Schedule) -> Schedule:
        """
        Lai tạo (Crossover) hai cá thể cha mẹ để tạo ra một cá thể con bằng
        phương pháp Point Crossover (tại một điểm cắt thời gian ngẫu nhiên).
        :param parent1: Cá thể cha mẹ 1.
        :param parent2: Cá thể cha mẹ 2.
        :return: Cá thể con mới (Schedule).
        """
        child = Schedule()
        child.schedule_data = defaultdict(list)

        # Chọn ngẫu nhiên một điểm cắt (slot_id)
        crossover_point = random.randint(0, TOTAL_SLOTS - 1)

        for slot_id in range(TOTAL_SLOTS):
            if slot_id < crossover_point:
                # Lấy gen từ parent1
                child.schedule_data[slot_id] = parent1.schedule_data[slot_id][:]
            else:
                # Lấy gen từ parent2
                child.schedule_data[slot_id] = parent2.schedule_data[slot_id][:]

        return child

    def mutate(self, schedule: Schedule, mutation_rate: float = 0.15):
        """
        Đột biến (Mutation): Thay đổi ngẫu nhiên vị trí của hai gen (bài giảng)
        giữa hai slot thời gian khác nhau để duy trì sự đa dạng.
        Đã sửa lỗi `pop from empty list` bằng cách chỉ chọn slot có chứa gen.
        :param schedule: Cá thể cần đột biến.
        :param mutation_rate: Tỷ lệ đột biến (xác suất xảy ra).
        """
        if random.random() < mutation_rate:

            # 1. Tìm tất cả các slot có chứa ít nhất một gen
            available_slots = [slot_id for slot_id, genes in schedule.schedule_data.items() if genes]

            # Kiểm tra nếu không đủ 2 slot để hoán đổi
            if len(available_slots) < 2:
                return

            # 2. Chọn ngẫu nhiên 2 slot KHÁC NHAU từ danh sách các slot có gen
            slot1_id, slot2_id = random.sample(available_slots, 2)

            # 3. Hoán đổi gen

            # Chọn ngẫu nhiên 1 gen từ slot 1 và pop nó (lấy ra và xóa)
            gene1_index = random.randint(0, len(schedule.schedule_data[slot1_id]) - 1)
            gene1 = schedule.schedule_data[slot1_id].pop(gene1_index)

            # Chọn ngẫu nhiên 1 gen từ slot 2 và pop nó
            gene2_index = random.randint(0, len(schedule.schedule_data[slot2_id]) - 1)
            gene2 = schedule.schedule_data[slot2_id].pop(gene2_index)

            # Hoán đổi gen (thêm gen vừa pop vào slot đối diện)
            schedule.schedule_data[slot1_id].append(gene2)
            schedule.schedule_data[slot2_id].append(gene1)

    def solve(self):
        """
        Chạy Giải thuật Di truyền qua các thế hệ.
        Quản lý vòng lặp tiến hóa, tính toán fitness, chọn lọc, lai tạo, đột biến
        và theo dõi cá thể tốt nhất (best_schedule).
        """
        self.initialize_population()

        for generation in range(self.generations):
            # 1. Tính toán Fitness cho toàn bộ quần thể
            for schedule in self.population:
                schedule.calculate_fitness()

            # 2. Lưu lại cá thể tốt nhất (Elitism)
            current_best = max(self.population, key=lambda s: s.fitness)

            if self.best_schedule is None or current_best.fitness > self.best_schedule.fitness:
                # Tạo bản sao sâu (deep copy) của cá thể tốt nhất
                self.best_schedule = self.crossover(current_best, current_best)
                self.best_schedule.calculate_fitness()

            # In thông tin thế hệ
            print(f"Thế hệ {generation+1}: Fitness tốt nhất = {self.best_schedule.fitness:.4f} (Penalty cứng: {self.best_schedule.hard_constraint_penalty})")

            # Điều kiện dừng sớm
            if self.best_schedule.hard_constraint_penalty == 0 and self.best_schedule.fitness >= 0.99:
                 print("\nĐã tìm thấy lịch trình tối ưu hoàn hảo. Dừng thuật toán.")
                 break

            # 3. Tạo quần thể mới
            new_population = []

            # Giữ lại cá thể tốt nhất (Elitism)
            new_population.append(self.best_schedule)

            while len(new_population) < self.population_size:
                # Chọn cha mẹ, Lai tạo, Đột biến để tạo con
                parent1, parent2 = self.select_parents()
                child = self.crossover(parent1, parent2)
                self.mutate(child, mutation_rate=0.15)
                new_population.append(child)

            self.population = new_population

        # Kết quả cuối cùng
        self.best_schedule.calculate_fitness()

        print("\n--- HOÀN THÀNH TỐI ƯU HÓA ---")
        self.best_schedule.display()


# --- 4. Chạy chương trình ---
if __name__ == "__main__":
    # Tham số GA
    POPULATION_SIZE = 150
    GENERATIONS = 200

    ga = GeneticAlgorithm(POPULATION_SIZE, GENERATIONS)
    ga.solve()

Thế hệ 1: Fitness tốt nhất = 1.0000 (Penalty cứng: 0)

Đã tìm thấy lịch trình tối ưu hoàn hảo. Dừng thuật toán.

--- HOÀN THÀNH TỐI ƯU HÓA ---
--- Lịch trình Tối ưu ---

### Lớp: 10A
| Tiết | **Thứ 2** | **Thứ 3** | **Thứ 4** | **Thứ 5** | **Thứ 6** |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **1** | (Trống) | (Trống) | (Trống) | (Trống) | (Trống) |
| **2** | Toán (GV_Toán_B) | Văn (GV_Văn_D) | (Trống) | (Trống) | (Trống) |
| **3** | Toán (GV_Toán_B) | Anh (GV_Anh_E) | Văn (GV_Văn_D) | Anh (GV_Anh_E) | Toán (GV_Toán_A) |
| **4** | Văn (GV_Văn_D) | (Trống) | (Trống) | Toán (GV_Toán_B) | (Trống) |
| **5** | (Trống) | Anh (GV_Anh_E) | (Trống) | Văn (GV_Văn_C) | (Trống) |

### Lớp: 10B
| Tiết | **Thứ 2** | **Thứ 3** | **Thứ 4** | **Thứ 5** | **Thứ 6** |
| :--- | :---: | :---: | :---: | :---: | :---: |
| **1** | Văn (GV_Văn_C) | (Trống) | (Trống) | Văn (GV_Văn_D) | (Trống) |
| **2** | (Trống) | (Trống) | (Trống) | Anh (GV_Anh_E) | Toán (GV_Toán_B) |
| **3** | (Trống) | (Trống) | (T

In [ ]:
# BÀI SỐ 3
import time

# ==============================================================================
# 0. Định nghĩa ngoại lệ cho Timeout
# ==============================================================================
class TimeoutException(Exception):
    """Ngoại lệ tùy chỉnh để ngắt quá trình tính toán Minimax khi hết thời gian."""
    pass

# ==============================================================================
# 1. CLASS: TicTacToe (Quản lý trạng thái và luật chơi)
# ==============================================================================
class TicTacToe:
    """
    Quản lý trạng thái bàn cờ, nước đi, và kiểm tra thắng thua cho game N x N (Cờ Caro/Gomoku).
    """
    def __init__(self, size, win_length=5):
        """
        Khởi tạo bàn cờ NxN.

        :param size: Kích thước bàn cờ (ví dụ: 5 cho 5x5).
        :param win_length: Số quân liên tiếp để thắng (K).
        """
        self.size = size
        self.win_length = win_length
        # ' ' là ô trống, 'X' là người chơi, 'O' là AI
        self.board = [[' ' for _ in range(size)] for _ in range(size)]
        self.current_player = 'X'  # 'X' (Người chơi) đi trước

    def display_board(self):
        """
        In bàn cờ ra màn hình với chỉ số hàng/cột (dễ dàng cho người dùng nhập liệu).
        """
        print("\n  " + " ".join([str(i) for i in range(self.size)]))
        print(" --" + "---" * self.size)
        for i in range(self.size):
            row_display = f"{i} | " + " | ".join(self.board[i]) + " |"
            print(row_display)
            print(" --" + "---" * self.size)

    def is_valid_move(self, row, col):
        """
        Kiểm tra nước đi có hợp lệ không.

        :param row: Chỉ số hàng.
        :param col: Chỉ số cột.
        :return: True nếu nước đi nằm trong phạm vi bàn cờ và ô đó còn trống, ngược lại False.
        """
        return 0 <= row < self.size and 0 <= col < self.size and self.board[row][col] == ' '

    def make_move(self, row, col, player):
        """
        Thực hiện nước đi trên bàn cờ.

        :param row: Chỉ số hàng.
        :param col: Chỉ số cột.
        :param player: Quân cờ ('X' hoặc 'O').
        :return: True nếu thực hiện thành công, False nếu nước đi không hợp lệ.
        """
        if self.is_valid_move(row, col):
            self.board[row][col] = player
            return True
        return False

    def check_win(self, player):
        """
        Kiểm tra xem người chơi 'player' đã thắng chưa (K quân liên tiếp).
        Quét tất cả các ô và kiểm tra 4 hướng: Ngang, Dọc, Chéo chính, Chéo phụ.

        :param player: Quân cờ ('X' hoặc 'O') cần kiểm tra.
        :return: True nếu player đã thắng, ngược lại False.
        """
        K = self.win_length
        N = self.size

        def check_line(r_start, c_start, dr, dc):
            """Hàm nội bộ kiểm tra một đường thẳng (theo hướng dr, dc)."""
            count = 0
            for i in range(N):
                r, c = r_start + i * dr, c_start + i * dc
                if 0 <= r < N and 0 <= c < N:
                    if self.board[r][c] == player:
                        count += 1
                        if count == K:
                            return True
                    else:
                        count = 0
                else:
                    break
            return False

        for r in range(N):
            for c in range(N):
                # Chỉ cần kiểm tra từ các ô có quân của người chơi
                if self.board[r][c] == player:
                    if check_line(r, c, 0, 1): return True    # Ngang
                    if check_line(r, c, 1, 0): return True    # Dọc
                    if check_line(r, c, 1, 1): return True    # Chéo chính (\)
                    if check_line(r, c, 1, -1): return True   # Chéo phụ (/)
        return False

    def is_board_full(self):
        """
        Kiểm tra xem bàn cờ đã đầy chưa (dẫn đến Hòa nếu không có ai thắng).

        :return: True nếu không còn ô trống nào, ngược lại False.
        """
        return all(self.board[r][c] != ' ' for r in range(self.size) for c in range(self.size))

    def get_game_status(self):
        """
        Trả về trạng thái hiện tại của trò chơi.

        :return: 'X_WINS', 'O_WINS', 'DRAW', hoặc 'CONTINUE'.
        """
        if self.check_win('X'):
            return 'X_WINS'
        if self.check_win('O'):
            return 'O_WINS'
        if self.is_board_full():
            return 'DRAW'
        return 'CONTINUE'

# ==============================================================================
# 2. CLASS: AIPlayer (Minimax với Alpha-Beta Pruning và Timeout)
# ==============================================================================
class AIPlayer:
    """
    Triển khai thuật toán Minimax với cắt tỉa Alpha-Beta để tìm nước đi tối ưu
    trong phạm vi độ sâu và thời gian cho phép.
    """
    def __init__(self, my_symbol, opponent_symbol, max_depth=2, timeout_seconds=5.0):
        """
        Khởi tạo AI Player.

        :param my_symbol: Quân cờ của AI (ví dụ: 'O').
        :param opponent_symbol: Quân cờ của đối thủ (ví dụ: 'X').
        :param max_depth: Độ sâu tối đa mà thuật toán Minimax sẽ tìm kiếm.
        :param timeout_seconds: Thời gian tối đa (giây) AI được phép tính toán cho một nước đi.
        """
        self.my_symbol = my_symbol
        self.opponent_symbol = opponent_symbol
        self.max_depth = max_depth
        self.timeout_seconds = timeout_seconds
        self.start_time = 0 # Dùng để lưu thời điểm bắt đầu tính toán

    def evaluate(self, game):
        """
        Hàm đánh giá heuristic. Gán điểm số cho trạng thái bàn cờ hiện tại.

        - Cung cấp điểm số rất lớn/rất nhỏ cho trạng thái thắng/thua.
        - Phân tích các chuỗi quân gần thắng (K-1, K-2) và cho điểm ưu tiên.
        (Đây là phần cần được cải tiến nhất để tăng độ mạnh của AI)

        :param game: Đối tượng TicTacToe hiện tại.
        :return: Điểm số heuristic (số nguyên), điểm càng cao càng có lợi cho AI.
        """
        if game.check_win(self.my_symbol):
            return 1000000000000
        if game.check_win(self.opponent_symbol):
            return -1000000000000

        score = 0
        K = game.win_length
        N = game.size

        def get_line_score(line):
            """Hàm nội bộ tính điểm cho một đoạn K ô liên tiếp."""
            s = 0
            my_count = line.count(self.my_symbol)
            opp_count = line.count(self.opponent_symbol)
            empty_count = line.count(' ')

            if my_count == K - 1 and empty_count >= 1:
                s += 10000 # Rất gần thắng
            if opp_count == K - 1 and empty_count >= 1:
                s -= 5000  # Nguy cơ thua

            if my_count == K - 2 and empty_count >= 2:
                 s += 100
            if opp_count == K - 2 and empty_count >= 2:
                 s -= 50

            return s

        # Quét tất cả các đoạn có độ dài K để tính tổng điểm heuristic
        for r in range(N):
            for c in range(N):
                # Quét 4 hướng và cộng dồn điểm
                if c <= N - K:
                    score += get_line_score([game.board[r][c+i] for i in range(K)])
                if r <= N - K:
                    score += get_line_score([game.board[r+i][c] for i in range(K)])
                if r <= N - K and c <= N - K:
                    score += get_line_score([game.board[r+i][c+i] for i in range(K)])
                if r <= N - K and c >= K - 1:
                    score += get_line_score([game.board[r+i][c-i] for i in range(K)])

        return score

    def minimax(self, game, depth, alpha, beta, is_maximizing_player):
        """
        Thuật toán Minimax đệ quy với Cắt tỉa Alpha-Beta.


        :param game: Trạng thái TicTacToe hiện tại.
        :param depth: Độ sâu còn lại để tìm kiếm.
        :param alpha: Giá trị Alpha (ngưỡng tối đa hiện tại cho người chơi MAX).
        :param beta: Giá trị Beta (ngưỡng tối thiểu hiện tại cho người chơi MIN).
        :param is_maximizing_player: True nếu là lượt của AI (MAX), False nếu là lượt đối thủ (MIN).
        :return: Điểm số tốt nhất tìm được từ trạng thái này.
        :raises TimeoutException: Nếu thời gian tính toán vượt quá giới hạn.
        """

        # ⚠️ LỆNH KIỂM TRA TIMEOUT
        if time.time() - self.start_time > self.timeout_seconds:
            # Ngừng ngay lập tức nếu quá thời gian
            raise TimeoutException("Thời gian tính toán đã hết.")

        # Trạng thái dừng (Base Case: độ sâu = 0 hoặc game kết thúc)
        status = game.get_game_status()
        if depth == 0 or status != 'CONTINUE':
            # Trả về điểm số tuyệt đối cho trạng thái thắng/thua, điểm heuristic cho trạng thái trung gian
            if status == f'{self.my_symbol}_WINS':
                return 100000000000000 + depth
            elif status == f'{self.opponent_symbol}_WINS':
                return -100000000000000 - depth
            elif status == 'DRAW':
                return 0
            return self.evaluate(game)

        possible_moves = [(r, c) for r in range(game.size) for c in range(game.size) if game.board[r][c] == ' ']

        if is_maximizing_player:
            max_eval = -float('inf')

            for r, c in possible_moves:
                game.make_move(r, c, self.my_symbol)
                eval = self.minimax(game, depth - 1, alpha, beta, False) # Chuyển sang lượt MIN
                game.board[r][c] = ' ' # Hoàn tác

                max_eval = max(max_eval, eval)
                alpha = max(alpha, max_eval)

                if beta <= alpha: # Cắt tỉa Beta
                    break

            return max_eval

        else: # is_maximizing_player is False (MIN Player)
            min_eval = float('inf')

            for r, c in possible_moves:
                game.make_move(r, c, self.opponent_symbol)
                eval = self.minimax(game, depth - 1, alpha, beta, True) # Chuyển sang lượt MAX
                game.board[r][c] = ' ' # Hoàn tác

                min_eval = min(min_eval, eval)
                beta = min(beta, min_eval)

                if beta <= alpha: # Cắt tỉa Alpha
                    break

            return min_eval

    def find_best_move(self, game):
        """
        Hàm chính để tìm nước đi tốt nhất cho AI.
        Thực hiện vòng lặp qua tất cả các nước đi hợp lệ và gọi Minimax để đánh giá.
        Có cơ chế bắt ngoại lệ Timeout để dừng tìm kiếm nếu quá lâu.

        :param game: Trạng thái TicTacToe hiện tại.
        :return: Tọa độ (hàng, cột) của nước đi tối ưu nhất.
        """
        self.start_time = time.time() # Bắt đầu tính thời gian
        best_eval = -float('inf')
        best_move = None

        possible_moves = [(r, c) for r in range(game.size) for c in range(game.size) if game.board[r][c] == ' ']

        if not possible_moves:
            return None, None

        # Tối ưu hóa: Nếu là nước đi đầu tiên, chọn trung tâm
        if len(possible_moves) == game.size * game.size:
            center = game.size // 2
            return center, center

        # Dùng Minimax để tìm kiếm
        try:
            for r, c in possible_moves:
                # Kiểm tra Timeout trước khi bắt đầu một nhánh tìm kiếm lớn
                if time.time() - self.start_time > self.timeout_seconds:
                    print(f"\n[CẢNH BÁO] Hết thời gian ({self.timeout_seconds}s). Chọn nước đi tốt nhất đã tìm được.")
                    break

                game.make_move(r, c, self.my_symbol)
                # Bắt đầu tìm kiếm từ độ sâu sâu nhất (-1 vì đã đi 1 nước)
                eval = self.minimax(game, self.max_depth - 1, -float('inf'), float('inf'), False)
                game.board[r][c] = ' '

                if eval > best_eval:
                    best_eval = eval
                    best_move = (r, c)

        except TimeoutException:
            print(f"\n[CẢNH BÁO] Hàm Minimax đã bị ngắt do Timeout ({self.timeout_seconds}s).")

        if best_move is None:
            # Trường hợp fallback: nếu bị timeout ngay từ move đầu tiên, chọn move đầu tiên trong danh sách
            return possible_moves[0]

        return best_move

# ==============================================================================
# 3. HÀM CHÍNH (Vòng lặp trò chơi và giao diện người dùng)
# ==============================================================================
def play_game():
    """
    Hàm chính điều khiển luồng trò chơi.
    - Lấy input từ người dùng (kích thước bàn cờ, số quân thắng, timeout).
    - Quản lý lượt chơi giữa người và AI.
    - Hiển thị bàn cờ và kết quả cuối cùng.
    """

    # --- Input Kích thước và Luật chơi ---
    while True:
        try:
            size_input = input("Nhập kích thước bàn cờ N x N (ví dụ: 5, 10): ")
            board_size = int(size_input)
            if board_size < 3:
                print("Kích thước bàn cờ phải >= 3.")
                continue
            break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    win_length_default = min(board_size, 5)
    while True:
        try:
            win_input = input(f"Nhập số quân liên tiếp để thắng K (Mặc định: {win_length_default}): ")
            if not win_input:
                win_length = win_length_default
            else:
                win_length = int(win_input)

            if win_length < 3 or win_length > board_size:
                print(f"Số quân liên tiếp để thắng phải nằm trong khoảng [3, {board_size}].")
                continue
            break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số nguyên.")

    # --- Thiết lập Độ sâu tìm kiếm và Timeout mặc định ---
    if board_size >= 7:
        search_depth = 1
        timeout_limit = 5.0
    elif board_size >= 5:
        search_depth = 2
        timeout_limit = 5.0
    else:
        search_depth = 4
        timeout_limit = 10.0

    # Lấy giới hạn thời gian từ người dùng
    while True:
        try:
            timeout_input = input(f"Nhập giới hạn thời gian tính toán của AI (giây) (Mặc định: {timeout_limit:.1f}s): ")
            if not timeout_input:
                ai_timeout = timeout_limit
            else:
                ai_timeout = float(timeout_input)
            if ai_timeout <= 0:
                print("Thời gian phải lớn hơn 0.")
                continue
            break
        except ValueError:
            print("Đầu vào không hợp lệ. Vui lòng nhập một số.")


    # --- Khởi tạo ---
    game = TicTacToe(size=board_size, win_length=win_length)
    ai = AIPlayer(my_symbol='O', opponent_symbol='X', max_depth=search_depth, timeout_seconds=ai_timeout)

    print(f"\n--- BẮT ĐẦU GAME CỜ CARO {board_size}x{board_size} | {win_length} quân để thắng ---")
    print(f"Bạn là 'X', AI là 'O'. Độ sâu tìm kiếm AI: {search_depth}. Giới hạn thời gian: {ai_timeout:.1f} giây.")

    # --- Vòng lặp Trò chơi Chính ---
    while game.get_game_status() == 'CONTINUE':
        game.display_board()

        # Lượt của Người chơi ('X')
        if game.current_player == 'X':
            # Xử lý nhập liệu của người dùng cho đến khi hợp lệ
            while True:
                try:
                    move_input = input("Lượt của X. Nhập [Hàng] [Cột] (ví dụ: 0 1): ")
                    r, c = map(int, move_input.split())
                    if game.make_move(r, c, 'X'):
                        game.current_player = 'O'
                        break
                    else:
                        print("Nước đi không hợp lệ (ngoài lề hoặc đã có quân). Thử lại.")
                except:
                    print("Đầu vào không hợp lệ. Vui lòng nhập 2 số nguyên cách nhau bằng khoảng trắng.")

        # Lượt của AI ('O')
        elif game.current_player == 'O':
            print("Lượt của AI (O) đang tính toán...")
            start_time = time.time()

            r_ai, c_ai = ai.find_best_move(game)

            end_time = time.time()

            if r_ai is None:
                # Thoát nếu AI không tìm được nước đi nào (trường hợp bàn cờ đầy)
                break

            print(f"AI đi: ({r_ai}, {c_ai}). Tổng thời gian tính: {end_time - start_time:.2f} giây")
            game.make_move(r_ai, c_ai, 'O')
            game.current_player = 'X'

    # --- Kết thúc Game ---
    game.display_board()
    status = game.get_game_status()
    print("\n==================================")
    if status == 'X_WINS':
        print("🎉 Người chơi (X) thắng! Chúc mừng bạn! 🎉")
    elif status == 'O_WINS':
        print("🤖 AI (O) thắng! Hãy thử lại! 🤖")
    else:
        print("🤝 Hòa! 🤝")
    print("==================================")

if __name__ == "__main__":
    play_game()

Nhập kích thước bàn cờ N x N (ví dụ: 5, 10): 3
Nhập số quân liên tiếp để thắng K (Mặc định: 3): 3
Nhập giới hạn thời gian tính toán của AI (giây) (Mặc định: 10.0s): 5

--- BẮT ĐẦU GAME CỜ CARO 3x3 | 3 quân để thắng ---
Bạn là 'X', AI là 'O'. Độ sâu tìm kiếm AI: 4. Giới hạn thời gian: 5.0 giây.

  0 1 2
 -----------
0 |   |   |   |
 -----------
1 |   |   |   |
 -----------
2 |   |   |   |
 -----------
Lượt của X. Nhập [Hàng] [Cột] (ví dụ: 0 1): 1 1

  0 1 2
 -----------
0 |   |   |   |
 -----------
1 |   | X |   |
 -----------
2 |   |   |   |
 -----------
Lượt của AI (O) đang tính toán...
AI đi: (0, 0). Tổng thời gian tính: 0.03 giây

  0 1 2
 -----------
0 | O |   |   |
 -----------
1 |   | X |   |
 -----------
2 |   |   |   |
 -----------
Lượt của X. Nhập [Hàng] [Cột] (ví dụ: 0 1): 0 1

  0 1 2
 -----------
0 | O | X |   |
 -----------
1 |   | X |   |
 -----------
2 |   |   |   |
 -----------
Lượt của AI (O) đang tính toán...
AI đi: (2, 1). Tổng thời gian tính: 0.01 giây

  0 1 2
 ---

1. Bài toán Người giao hàng (TSP) bằng Thuật toán A*

Bài toán TSP yêu cầu tìm lộ trình ngắn nhất đi qua một danh sách các thành phố, mỗi thành phố đúng một lần và quay trở lại điểm xuất phát.

Cấu trúc và Logic triển khai

Lớp TSPNode: Đại diện cho một trạng thái trong cây tìm kiếm4. Nó lưu trữ lộ trình hiện tại (path), chi phí đã đi (cost), thành phố hiện tại và tập hợp các thành phố chưa ghé thăm.

Hàm Heuristic (n): Sử dụng phương pháp "cạnh ngắn nhất" để ước lượng chi phí còn lại. Cụ thể, $h(n)$  bằng tổng các cạnh ngắn nhất rời khỏi từng thành phố chưa ghé thăm cộng với khoảng cách từ thành phố hiện hành quay về điểm bắt đầu.

Thuật toán tìm kiếm: Sử dụng hàng đợi ưu tiên (heapq) để quản lý các nút. Thuật toán luôn mở rộng nút có giá trị $f = g + h$ nhỏ nhất, đảm bảo tính tối ưu của lời giải.

Kết quả thực thi

Hệ thống xử lý nhập liệu từ người dùng cho ma trận khoảng cách (có thể là đồ thị có hướng) 10. Ví dụ với 3 thành phố, thuật toán đã tìm ra chu trình ngắn nhất $0 \to 1 \to 2 \to 0$ với tổng chi phí là 8.0 .

2. Lập lịch Thời khóa biểu bằng Giải thuật Di truyền (GA)

Bài toán này giải quyết việc sắp xếp lịch học cho các lớp sao cho không bị trùng lặp giáo viên và thỏa mãn các yêu cầu về số tiết học.

Cơ chế tiến hóa

Cá thể (Chromosome): Đại diện bởi lớp Schedule, lưu trữ dữ liệu dưới dạng các khung giờ (slots) trong tuần. Mỗi slot chứa thông tin về (Lớp, Môn, Giáo viên).

Hàm thích nghi (Fitness):

Ràng buộc cứng: Kiểm tra trùng lịch giáo viên hoặc trùng lịch lớp. Nếu vi phạm, độ thích nghi giảm mạnh theo công thức $Fitness = \frac{1.0}{1.0 + penalty \times 1000}$.

Ràng buộc mềm: Ưu tiên phân bổ đều môn học (không quá 2 tiết/môn/ngày) và giảm thiểu các tiết trống (idle slots) của giáo viên.

Các phép toán di truyền:

Chọn lọc: Sử dụng Tournament Selection (chọn cá thể tốt nhất từ một nhóm ngẫu nhiên) .

Lai tạo: Point Crossover - đổi các đoạn mã gen (lịch học) giữa hai cha mẹ tại một điểm cắt ngẫu nhiên.

Đột biến: Hoán đổi ngẫu nhiên các tiết học giữa hai khung giờ khác nhau để tránh rơi vào tối ưu cục bộ.

Kết quảChương trình thực hiện qua nhiều thế hệ và áp dụng chiến lược Elitism (giữ lại cá thể tốt nhất cho thế hệ sau). Kết quả cuối cùng xuất ra bảng lịch trình chi tiết cho từng lớp học dưới dạng bảng Markdown trực.

3. Trò chơi Caro (TicTacToe) N x N bằng Minimax

Chương trình cung cấp một môi trường chơi cờ Caro với kích thước bàn cờ tùy chỉnh và khả năng đối kháng với AI.Đặc điểm kỹ thuậtThuật toán Minimax & Alpha-Beta: AI tìm kiếm nước đi tối ưu bằng cách giả lập các bước đi trong tương lai. Cắt tỉa Alpha-Beta được áp dụng để loại bỏ các nhánh không cần thiết, giúp tăng tốc độ xử lý trên bàn cờ lớn.

Hàm đánh giá Heuristic: Gán điểm cho các chuỗi quân cờ . Chuỗi gần thắng ($K-1$ quân và có ô trống) sẽ được ưu tiên cao ($+10,000$ điểm), trong khi các mối đe dọa từ đối thủ sẽ bị trừ điểm nặng.

Quản lý hiệu năng:

Timeout: Tự động ngắt tìm kiếm nếu vượt quá giới hạn thời gian quy định (ví dụ: 5.0 giây) và trả về nước đi tốt nhất tìm được tính đến thời điểm đó.

Nước đi đầu: Nếu AI đi đầu tiên, nó sẽ tự động chọn vị trí trung tâm để tối ưu hóa thế trận.

Giao diện trò chơiNgười dùng có thể tùy chỉnh kích thước $N$, số quân thắng $K$ và thời gian AI suy nghĩ. AI hiển thị thời gian tính toán thực tế sau mỗi nước đi (thường chỉ mất từ 0.00s đến 0.03s cho bàn cờ 3x3).

# TRI-TUE-NHA-TAO1: Các Thuật Toán AI Cơ Bản

**Dự án tổng hợp và triển khai ba bài toán điển hình trong lĩnh vực Trí tuệ Nhân tạo: Tìm kiếm Tối ưu (A\*), Tối ưu hóa (Giải thuật Di truyền), và Game Theory (Minimax).**

## Mục lục

1.  Giới thiệu
2.  Các Bài toán Đã Giải quyết
      * Bài 1: Bài toán Người Giao Hàng (TSP) bằng A\*
      * Bài 2: Xếp Lịch Tự Động bằng Giải thuật Di truyền (GA)
      * Bài 3: Game Cờ Caro N x N bằng Minimax với Alpha-Beta Pruning
3.  Yêu cầu Hệ thống & Thiết lập
4.  Hướng dẫn Sử dụng
5.  Tác giả

## Giới thiệu

Dự án này là một báo cáo/notebook (`Bai_bao_cao.ipynb`) nhằm triển khai các thuật toán tìm kiếm và tối ưu hóa cổ điển, thể hiện khả năng giải quyết các vấn đề phức tạp trong AI.

## Các Bài toán Đã Giải quyết

### Bài 1: Bài toán Người Giao Hàng (TSP) bằng A\*

  * **Mục tiêu:** Tìm chu trình ngắn nhất đi qua tất cả các thành phố và quay về điểm xuất phát (Traveling Salesperson Problem).
  * **Thuật toán:** A\* Search.
  * **Triển khai:**
      * Sử dụng lớp `TSP_AStar` với trạng thái `TSPNode`.
      * Hàm Heuristic (`h(n)`) dựa trên **Cận Dưới (Lower Bound)**: Tính tổng các cạnh ngắn nhất rời khỏi mỗi thành phố chưa ghé thăm, cộng với cạnh từ thành phố hiện tại về điểm xuất phát.
  * **Lưu ý:** A\* giải TSP là bài toán NP-hard, do đó chương trình có thể chạy chậm với số lượng thành phố lớn (N \> 10-12).

### Bài 2: Xếp Lịch Tự Động bằng Giải thuật Di truyền (GA)

  * **Mục tiêu:** Xây dựng lịch trình dạy học tối ưu cho nhiều lớp/giáo viên, thỏa mãn một tập hợp các ràng buộc cứng và mềm.
  * **Thuật toán:** Genetic Algorithm (Giải thuật Di truyền).
  * **Triển khai:**
      * **Cá thể (`Schedule`):** Đại diện cho một lịch trình hoàn chỉnh.
      * **Độ thích nghi (Fitness):** Được tính toán dựa trên mức độ vi phạm các ràng buộc.
          * **Ràng buộc Cứng:** Trùng lịch Giáo viên và Trùng lịch Lớp học (bị phạt nặng).
          * **Ràng buộc Mềm:** Phân bố tiết học hợp lý (không quá 2 tiết cùng môn/ngày) và giảm thiểu giờ rảnh rỗi của giáo viên (Teacher Idle Slots).
      * **Cơ chế Tiến hóa:** Sử dụng Tournament Selection, Point Crossover và Đột biến bằng cách hoán đổi ngẫu nhiên vị trí của hai tiết học.

### Bài 3: Game Cờ Caro N x N bằng Minimax với Alpha-Beta Pruning

  * **Mục tiêu:** Tạo một đối thủ AI mạnh cho trò chơi Cờ Caro ($N \times N$ với $K$ quân liên tiếp để thắng).
  * **Thuật toán:** Minimax với Cắt tỉa Alpha-Beta.
  * **Triển khai:**
      * **Lớp `AIPlayer`:** Tìm kiếm nước đi tối ưu.
      * **Hàm Đánh giá (`evaluate`):** Phân tích các chuỗi quân gần thắng (`K-1`, `K-2`) để đưa ra điểm heuristic.
      * **Tính năng Đặc biệt:**
          * Cơ chế **Timeout** để ngắt quá trình tìm kiếm nếu vượt quá giới hạn thời gian (quan trọng cho bàn cờ lớn).
          * Cho phép người dùng tùy chỉnh kích thước bàn cờ ($N$), số quân thắng ($K$), và giới hạn thời gian AI.

## Yêu cầu Hệ thống & Thiết lập

  * **Ngôn ngữ:** Python
  * **Môi trường:** Jupyter Notebook hoặc Google Colab.
  * **Thư viện cơ bản:** `heapq`, `math`, `random`, `collections` (đã được sử dụng trong mã nguồn).

## Hướng dẫn Sử dụng

1.  Mở file `Bai_bao_cao.ipynb` bằng Google Colab (có sẵn nút **"Open in Colab"** trên GitHub) hoặc trong môi trường Jupyter Notebook.
2.  Chạy tuần tự các khối mã (cell) trong Notebook.
3.  **Tương tác với chương trình:**
      * **Bài 1 (TSP/A\*):** Chương trình sẽ chạy trong terminal và yêu cầu nhập số lượng thành phố ($N$) và ma trận khoảng cách.
      * **Bài 2 (GA/Xếp Lịch):** Thuật toán sẽ chạy tự động và hiển thị quá trình tiến hóa (`Fitness tốt nhất`).
      * **Bài 3 (Cờ Caro/Minimax):** Chương trình sẽ yêu cầu nhập các tham số (kích thước $N$, $K$, Timeout) và bắt đầu vòng lặp chơi giữa Người chơi (`X`) và AI (`O`).

## Tác giả

  * Repository: [thuh66271-arch](https://www.google.com/search?q=https://github.com/thuh66271-arch)